In [124]:
# %pip install hvplot
# %pip install plotly
# %pip install holoviews
# %pip install xarray
# %pip install panel

In [125]:
import numpy as np

import datetime as dt
import panel as pn

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')

In [126]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"

# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"

In [127]:
from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter

In [128]:
display(f'{client.version=} / {client.config.project=}')
# print(client.iam.token.inspect())

"client.version='6.39.6' / client.config.project='equinor-dev'"

In [129]:
#test

In [130]:
wellcom_dataset = "src:015:wellcom"
edm_dataset = "src:012:edm"
witsml_dataset = "src:014:witsmldata"

In [131]:
active_wellbore = "active:wellbore"
match_witsml = "match:witsml:wellcom" # sensor data only for johan svendrup
match_edm = "match:edm:wellcom"

# defien and show datasets

In [132]:
#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
active_wellbores

,external_id,name,parent_id,parent_external_id,description,data_set_id,metadata,source,labels,id,created_time,last_updated_time,root_id
0,dw:wellcom:WELLBORE:164477857,NO 16/5-K-3 H,7876942777586616,dw:wellcom:WELL:164477851,Wellbore information for: NO 16/5-K-3 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",955508372036290,1694032030374,1715821671591,7876942777586616
1,dw:wellcom:WELLBORE:174967378,NO 16/5-K-2 H,1827766739511413,dw:wellcom:WELL:174967353,Wellbore information for: NO 16/5-K-2 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:witsml:wellcom'}, {'ext...",1713830288579674,1694032030530,1715821671591,1827766739511413
2,dw:wellcom:WELLBORE:171031132,NO 16/3-O-3 H,3526246276582693,dw:wellcom:WELL:171030413,Wellbore information for: NO 16/3-O-3 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",2332471204802633,1694032030764,1710723960939,3526246276582693
3,dw:wellcom:WELLBORE:94103429,NO 16/2-D-1,5785855130534576,dw:wellcom:WELL:93930579,Wellbore information for: NO 16/2-D-1,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",7215111947073855,1694032030858,1710723957890,5785855130534576


In [133]:
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)
witsml_db

,id,external_id,name,is_string,metadata,unit,asset_id,is_step,description,security_categories,data_set_id,created_time,last_updated_time
0,881650803765,dw:witsml:log.49a36410-e7fb-4864-b3c7-c1eba937...,CEMPUMPPR,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,2033175562258369,False,CEMPUMPPR,[],7412963440371515,1699923727680,1718715610208
1,1641460701259,dw:witsml:log.4eccc70b-adba-442b-852c-8f1ab7cb...,MROUT,False,"{'type': 'log', 'commonData_dTimCreation': '20...",ohm.m,8564953405341906,False,MROUT,[],7412963440371515,1699923657815,1718715610208
2,2811296096383,dw:witsml:log.e17da3e3-6e4f-4fdb-a53b-3a433927...,CEMWTIN,False,"{'type': 'log', 'commonData_dTimCreation': '20...",kg/m3,8568419009254753,False,CEMWTIN,[],7412963440371515,1699923671389,1718715610208
3,3562630500496,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,PIT30,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m3,2333496463722180,False,PIT30,[],7412963440371515,1699923753693,1718715610208
4,4349328030238,dw:witsml:log.e5912051-eec4-4b18-9dd5-bc687894...,KILL_PRESS,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,6803620337242231,False,KILL_PRESS,[],7412963440371515,1699923718702,1718715610208
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,9003442841042893,dw:witsml:log.e6caf2f9-3e59-4e5d-8b63-b1f495bd...,MWOUT,False,"{'type': 'log', 'commonData_dTimCreation': '20...",kg/m3,4886106907570570,False,MWOUT,[],7412963440371515,1699923682072,1718715612331
9479,9004567224869856,dw:witsml:log.c54bfb78-c126-4982-94b1-6102b1eb...,MUDRETDEPTH,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m,8081963593989476,False,MUDRETDEPTH,[],7412963440371515,1699923743864,1718715612331
9480,9004904195500575,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,CHOKE_PRESS,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,1783617027716013,False,CHOKE_PRESS,[],7412963440371515,1699923739422,1718715612331
9481,9005083314734865,dw:witsml:log.7bd886cb-97e4-4487-92fe-2e1e1f45...,ROP_AVG,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m/s,8725854939492372,False,ROP_AVG,[],7412963440371515,1699923741532,1718715612331


## make to padnas

In [134]:
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)
pd_active_wellbores

,external_id,name,parent_id,parent_external_id,description,data_set_id,metadata,source,labels,id,created_time,last_updated_time,root_id
0,dw:wellcom:WELLBORE:164477857,NO 16/5-K-3 H,7876942777586616,dw:wellcom:WELL:164477851,Wellbore information for: NO 16/5-K-3 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",955508372036290,1694032030374,1715821671591,7876942777586616
1,dw:wellcom:WELLBORE:174967378,NO 16/5-K-2 H,1827766739511413,dw:wellcom:WELL:174967353,Wellbore information for: NO 16/5-K-2 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:witsml:wellcom'}, {'ext...",1713830288579674,1694032030530,1715821671591,1827766739511413
2,dw:wellcom:WELLBORE:171031132,NO 16/3-O-3 H,3526246276582693,dw:wellcom:WELL:171030413,Wellbore information for: NO 16/3-O-3 H,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",2332471204802633,1694032030764,1710723960939,3526246276582693
3,dw:wellcom:WELLBORE:94103429,NO 16/2-D-1,5785855130534576,dw:wellcom:WELL:93930579,Wellbore information for: NO 16/2-D-1,6346490050989733,"{'EXISTENCE_KD_NM': 'ACTUAL', 'LAST_DR_DATE': ...",src:015:wellcom:rawdb.wellbore,"[{'externalId': 'match:edm:wellcom'}, {'extern...",7215111947073855,1694032030858,1710723957890,5785855130534576


In [135]:
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml

,id,external_id,name,is_string,metadata,unit,asset_id,is_step,description,security_categories,data_set_id,created_time,last_updated_time
0,881650803765,dw:witsml:log.49a36410-e7fb-4864-b3c7-c1eba937...,CEMPUMPPR,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,2033175562258369,False,CEMPUMPPR,[],7412963440371515,1699923727680,1718715610208
1,1641460701259,dw:witsml:log.4eccc70b-adba-442b-852c-8f1ab7cb...,MROUT,False,"{'type': 'log', 'commonData_dTimCreation': '20...",ohm.m,8564953405341906,False,MROUT,[],7412963440371515,1699923657815,1718715610208
2,2811296096383,dw:witsml:log.e17da3e3-6e4f-4fdb-a53b-3a433927...,CEMWTIN,False,"{'type': 'log', 'commonData_dTimCreation': '20...",kg/m3,8568419009254753,False,CEMWTIN,[],7412963440371515,1699923671389,1718715610208
3,3562630500496,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,PIT30,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m3,2333496463722180,False,PIT30,[],7412963440371515,1699923753693,1718715610208
4,4349328030238,dw:witsml:log.e5912051-eec4-4b18-9dd5-bc687894...,KILL_PRESS,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,6803620337242231,False,KILL_PRESS,[],7412963440371515,1699923718702,1718715610208
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,9003442841042893,dw:witsml:log.e6caf2f9-3e59-4e5d-8b63-b1f495bd...,MWOUT,False,"{'type': 'log', 'commonData_dTimCreation': '20...",kg/m3,4886106907570570,False,MWOUT,[],7412963440371515,1699923682072,1718715612331
9479,9004567224869856,dw:witsml:log.c54bfb78-c126-4982-94b1-6102b1eb...,MUDRETDEPTH,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m,8081963593989476,False,MUDRETDEPTH,[],7412963440371515,1699923743864,1718715612331
9480,9004904195500575,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,CHOKE_PRESS,False,"{'type': 'log', 'commonData_dTimCreation': '20...",Pa,1783617027716013,False,CHOKE_PRESS,[],7412963440371515,1699923739422,1718715612331
9481,9005083314734865,dw:witsml:log.7bd886cb-97e4-4487-92fe-2e1e1f45...,ROP_AVG,False,"{'type': 'log', 'commonData_dTimCreation': '20...",m/s,8725854939492372,False,ROP_AVG,[],7412963440371515,1699923741532,1718715612331


### query pandas witsml

In [136]:
_pd_witsml.columns

Index(['id', 'external_id', 'name', 'is_string', 'metadata', 'unit',
       'asset_id', 'is_step', 'description', 'security_categories',
       'data_set_id', 'created_time', 'last_updated_time'],
      dtype='object')

In [137]:
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

## flatten witsml data

In [138]:
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'])], axis=1)



In [139]:
_pd_witsml_flattened.columns

Index(['id', 'external_id', 'name', 'is_string', 'unit', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType', 'name', 'nameWell',
       'nameWellbore', 'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex', 'uidWell', 'uidWellbore', 'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid', 'unit'],
      dtype='object')

In [140]:
_pd_witsml_flattened = _pd_witsml_flattened[['id', 'uidWell', 'uidWellbore',  'nameWellbore','external_id', 'name', 'is_string', 'unit', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType', 'name', 'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid', 'unit']]
_pd_witsml_flattened

,id,uidWell,uidWellbore,nameWellbore,external_id,name,name,is_string,unit,unit,...,source,curveDescription,maxDateTimeIndex,minDateTimeIndex,mnemAlias,mnemonic,typeLogData,uid,unit,unit
0,881650803765,49a36410-e7fb-4864-b3c7-c1eba937dcd9,0ac9cfa3-b6a0-4af3-85e4-3180f68f0c11,NO 16/2-D-2 T4,dw:witsml:log.49a36410-e7fb-4864-b3c7-c1eba937...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2023-01-11T17:29:47.000000Z,2022-11-24T05:27:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
1,1641460701259,4eccc70b-adba-442b-852c-8f1ab7cbcf77,b467d241-4086-47c5-9c6a-a6fc8cd010ce,NO 16/2-D-10,dw:witsml:log.4eccc70b-adba-442b-852c-8f1ab7cb...,MROUT,Surface Logging Data - Time RT,False,ohm.m,ohm.m,...,KongsbergDigital,Mud Resistivity Out,2019-08-22T08:40:57.000000Z,2019-08-22T03:56:45.000000Z,MROUT,MROUT,double,MROUT,ohm.m,ohm.m
2,2811296096383,e17da3e3-6e4f-4fdb-a53b-3a43392721ef,dc0ea5a6-c45c-4e1b-a57b-cde2bdd3af2c,NO 16/2-D-42,dw:witsml:log.e17da3e3-6e4f-4fdb-a53b-3a433927...,CEMWTIN,Surface Logging Data - Time RT,False,kg/m3,kg/m3,...,KongsbergDigital,Cementing Fluid Density In,2021-04-30T18:29:10.000000Z,2020-06-04T08:59:00.000000Z,CEMWTIN,CEMWTIN,double,CEMWTIN,kg/m3,kg/m3
3,3562630500496,aa2787e0-bd34-40ba-9783-6426a3d0f952,542e790e-9b76-40a6-ab97-3b75e1ebfa8c,NO 16/2-H-3 AH,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,PIT30,Surface Logging Data - Time RT,False,m3,m3,...,KongsbergDigital,Pit 30 Volume,2023-07-28T20:23:30.000000Z,2023-07-09T20:56:30.000000Z,PIT30,PIT30,double,PIT30,m3,m3
4,4349328030238,e5912051-eec4-4b18-9dd5-bc687894a406,b2598fc0-bcce-4120-a51c-5006ff331fad,NO 16/2-D-37,dw:witsml:log.e5912051-eec4-4b18-9dd5-bc687894...,KILL_PRESS,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Kill Pressure,2022-10-12T08:05:02.000000Z,2022-02-06T01:26:18.000000Z,KILL_PRESS,KILL_PRESS,double,KILL_PRESS,Pa,Pa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,9003442841042893,e6caf2f9-3e59-4e5d-8b63-b1f495bd4cab,76a03464-fc18-4fac-9e36-64d683334a78,NO 16/2-D-17 T2,dw:witsml:log.e6caf2f9-3e59-4e5d-8b63-b1f495bd...,MWOUT,Surface Logging Data - Time RT,False,kg/m3,kg/m3,...,KongsbergDigital,Mud Weight Out,2021-03-06T23:31:28.000000Z,2021-02-12T16:46:14.000000Z,MWOUT,MWOUT,double,MWOUT,kg/m3,kg/m3
9479,9004567224869856,c54bfb78-c126-4982-94b1-6102b1ebfbc4,ae0d9b30-14d1-4b4c-b259-1924f241e304,NO 16/3-Q-4 H,dw:witsml:log.c54bfb78-c126-4982-94b1-6102b1eb...,MUDRETDEPTH,Surface Logging Data - Time RT,False,m,m,...,KongsbergDigital,Lagged Mud Returns Depth,2023-12-01T20:00:00.000000Z,2023-06-20T22:31:00.000000Z,MUDRETDEPTH,MUDRETDEPTH,double,MUDRETDEPTH,m,m
9480,9004904195500575,aa2787e0-bd34-40ba-9783-6426a3d0f952,96d15724-b9f8-4d38-a392-5b585cb9900a,NO 16/2-H-3 H,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,CHOKE_PRESS,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Choke Pressure,2023-07-09T20:43:59.000000Z,2023-06-11T18:32:42.000000Z,CHOKE_PRESS,CHOKE_PRESS,double,CHOKE_PRESS,Pa,Pa
9481,9005083314734865,7bd886cb-97e4-4487-92fe-2e1e1f4515a7,7ce59fac-8b44-4176-8c4d-60f09dc89de3,NO 16/3-Q-2 H,dw:witsml:log.7bd886cb-97e4-4487-92fe-2e1e1f45...,ROP_AVG,Surface Logging Data - Time RT,False,m/s,m/s,...,KongsbergDigital,Time Averaged ROP,2023-06-20T17:39:47.000000Z,2023-06-17T12:28:52.000000Z,ROP_AVG,ROP_AVG,double,ROP_AVG,m/s,m/s


In [141]:
_pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']['external_id'].iloc[0]

'dw:witsml:log.90485438-0aba-42a5-bc78-75e3a21bcf72.5a8d0cff-5299-4bde-bc4f-1fcffa81be50.5a8d0cff-5299-4bde-bc4f-1fcffa81be50.bitdep||v1'

In [142]:
_pd_witsml_flattened['nameWell'].nunique()

56

In [143]:
_df_name_is_cempumppr = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'CEMPUMPPR']

In [144]:
_df_name_is_cempumppr.sort_values(by=['startDateTimeIndex']).head(80)

,id,uidWell,uidWellbore,nameWellbore,external_id,name,name,is_string,unit,unit,...,source,curveDescription,maxDateTimeIndex,minDateTimeIndex,mnemAlias,mnemonic,typeLogData,uid,unit,unit
6797,6412144277762574,c54bfb78-c126-4982-94b1-6102b1ebfbc4,ae0d9b30-14d1-4b4c-b259-1924f241e304,NO 16/3-Q-4 H,dw:witsml:log.c54bfb78-c126-4982-94b1-6102b1eb...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2023-06-20T22:46:20.000000Z,1900-01-01T00:00:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
9089,8616102998055654,0cf8f65b-cc3a-4608-bfcd-615214a2ee4a,96ab3a86-5e15-442f-ba3a-e9ae7a494ea2,NO 16/2-D-18,dw:witsml:log.0cf8f65b-cc3a-4608-bfcd-615214a2...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2020-02-21T17:02:00.000000Z,2019-12-22T19:46:40.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
2175,2008009588366199,f5994408-36e6-4143-8b83-d04def72f951,14ad7b6b-4b96-4581-a1c5-4490b0a5c935,NO 16/2-D-14,dw:witsml:log.f5994408-36e6-4143-8b83-d04def72...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2019-06-07T07:44:30.000000Z,2019-05-29T08:54:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
5269,5007980212020715,8ae2ee08-9235-4e02-9818-15b223232380,ab67aae3-4235-4bca-acea-3079c48824b5,NO 16/2-D-12,dw:witsml:log.8ae2ee08-9235-4e02-9818-15b22323...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2019-10-08T08:19:30.000000Z,2019-06-07T09:25:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
3018,2813543266430008,e6caf2f9-3e59-4e5d-8b63-b1f495bd4cab,8b1fbf9e-c8aa-4e35-9c3c-8bab417e43fa,NO 16/2-D-17,dw:witsml:log.e6caf2f9-3e59-4e5d-8b63-b1f495bd...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2021-02-12T15:31:40.000000Z,2021-01-26T06:55:10.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,2562056459241702,11a065ca-4325-4da0-ab9b-07a288dd97e1,c9587688-bfe5-4289-b221-e9270b1ddd0d,NO 16/2-D-48,dw:witsml:log.11a065ca-4325-4da0-ab9b-07a288dd...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2024-02-13T22:49:10.000000Z,2024-02-09T18:14:11.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
1548,1439318167596414,95992c20-f691-40d9-86e1-4f3e5ce823b4,51c884c6-bafa-47c2-9bf2-e7785a2d2c01,NO 16/2-D-35 T2,dw:witsml:log.95992c20-f691-40d9-86e1-4f3e5ce8...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2024-03-25T11:59:50.000000Z,2024-02-18T05:23:42.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
5027,4759913348357306,b198ca8c-58f2-490b-91cf-797ac6079d28,ef3d755c-c00f-43ec-8e6a-4d6b9438168f,NO 16/3-O-1 HT2,dw:witsml:log.b198ca8c-58f2-490b-91cf-797ac607...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2024-04-25T09:16:58.000000Z,2024-03-31T23:17:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa
1493,1385525806544474,2f1add5d-dc06-4be1-94db-211670a07c84,01f5a477-a501-4e43-b80d-17f034b5f254,NO 16/2-D-44 T2,dw:witsml:log.2f1add5d-dc06-4be1-94db-211670a0...,CEMPUMPPR,Surface Logging Data - Time RT,False,Pa,Pa,...,KongsbergDigital,Cementing Pump Pressure,2024-06-10T14:10:50.000000Z,2024-05-10T16:52:30.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR,Pa,Pa


In [145]:
len(_df_name_is_cempumppr)

78

In [146]:
len(_df_name_is_cempumppr['nameWellbore'].unique())

74

## find sensorid to base real time data on

In [147]:
_df_name_is_bitdep = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']

In [148]:
_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-18']

,id,uidWell,uidWellbore,nameWellbore,external_id,name,name,is_string,unit,unit,...,source,curveDescription,maxDateTimeIndex,minDateTimeIndex,mnemAlias,mnemonic,typeLogData,uid,unit,unit
7519,7119861774595595,0cf8f65b-cc3a-4608-bfcd-615214a2ee4a,96ab3a86-5e15-442f-ba3a-e9ae7a494ea2,NO 16/2-D-18,dw:witsml:log.0cf8f65b-cc3a-4608-bfcd-615214a2...,BITDEP,Surface Logging Data - Time RT,False,m,m,...,KongsbergDigital,The mnemonic of the Hole depth,2020-02-21T17:02:06.000000Z,2019-04-30T08:47:35.000000Z,BITDEP,BITDEP,double,BITDEP,m,m


## set sensorid

In [149]:
_sel_sensorid = int(_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-15'].id.iloc[0])
_sel_sensorid 

7754896165576453

#### make list of all sensorids to use in panel

In [150]:
_sensorids_list = pd.Series(_df_name_is_bitdep['nameWellbore']).unique().tolist()
# _sensorids_list

In [151]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list


{114925988185573: 'NO 16/3-O-2 HT2',
 540388646753556: 'NO 16/2-D-44',
 553303436762434: 'NO 16/2-D-42 T2',
 610889273075352: 'NO 16/2-H-3 H',
 746172675242144: 'NO 16/2-D-43',
 807541165501966: 'NO 16/3-P-4 H',
 864186469645318: 'NO 16/3-Q-1 H',
 885149193008514: 'NO 16/2-D-7',
 908832947568610: 'NO 16/2-D-4',
 956495648801814: 'NO 16/3-Q-4 HT2',
 1138197140994517: 'NO 16/2-D-42',
 1415941041496504: 'NO 16/2-D-40',
 1570677098860114: 'NO 16/2-D-37',
 1743715413643747: 'NO 16/3-O-4 H',
 1820004034410290: 'NO 16/3-O-1 HT2',
 1822774244156829: 'NO 16/3-Q-4 H',
 1934667450362902: 'NO 16/3-O-2 HT3',
 2086215955982590: 'NO 16/2-D-1',
 2181584989341830: 'NO 16/2-D-2 T2',
 2220634486483244: 'NO 16/2-D-10',
 2337663974754072: 'NO 16/2-D-39',
 2452199672533487: 'NO 16/2-D-21',
 2759820873419026: 'NO 16/2-D-41',
 2832860854551760: 'NO 16/2-D-38',
 2837910117511181: 'NO 16/2-D-16 T2',
 2896995270371612: 'NO 16/2-H-1 H',
 2911505954954534: 'NO 16/5-K-2 H',
 3088543739857874: 'NO 16/2-D-17 T2',
 31

#### panel choose _sel_sensorid

In [152]:
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')

# def update_indicator(event, clicks):
#     if not event:
#         return
    
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"

# pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)

# pn.Column(pn.Row(test_widget, text),indicator)

In [153]:
# _sensorids_list = [{1:"fefe"}, {2 :"foo"}]

In [154]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list



_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid( self, value):
    _sel_sensorid = value
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'de34b97d-5241-4c11-9758-7802a05d5966': {'version…

In [155]:
_sel_sensorid =_sel_sensorid_widget.param.values().__getitem__('value')
_sel_sensorid
## TODO why is _sel_sensorid not updating automatically?

'NO 16/3-O-2 HT2'

In [156]:
_sel_sensorid

'NO 16/3-O-2 HT2'

In [157]:
_sel_sensorid = 746172675242144

## TIME SERIES QUERY based on _sel_sensorid

In [158]:
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

In [159]:
client.time_series.data.retrieve(id=746172675242144,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1)

,dw:witsml:log.5a07f5ab-b3e0-4bd1-8a5d-8a1a72a75987.7c135cc1-5b81-4ef7-be77-adad018e5e17.7c135cc1-5b81-4ef7-be77-adad018e5e17.bitdep||v1
2022-01-26 07:42:42,2.0
2022-01-26 07:42:43,2.0
2022-01-26 07:42:44,2.0
2022-01-26 07:42:45,2.0
2022-01-26 07:42:46,2.0
...,...
2022-11-15 11:35:40,0.0
2022-11-15 11:35:41,0.0
2022-11-15 11:35:42,0.0
2022-11-15 11:35:43,0.0


In [160]:
_df_real_time = client.time_series.data.retrieve(
    id=_sel_sensorid,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1) 
_df_real_time


,dw:witsml:log.5a07f5ab-b3e0-4bd1-8a5d-8a1a72a75987.7c135cc1-5b81-4ef7-be77-adad018e5e17.7c135cc1-5b81-4ef7-be77-adad018e5e17.bitdep||v1
2022-01-26 07:42:42,2.0
2022-01-26 07:42:43,2.0
2022-01-26 07:42:44,2.0
2022-01-26 07:42:45,2.0
2022-01-26 07:42:46,2.0
...,...
2022-11-15 11:35:40,0.0
2022-11-15 11:35:41,0.0
2022-11-15 11:35:42,0.0
2022-11-15 11:35:43,0.0


## to pandas

In [161]:
_pd_real_time = _df_real_time.to_pandas( )
# make Value columsn or easier filtering
_pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]

In [162]:
_pd_real_time.sort_index(ascending=True)

,dw:witsml:log.5a07f5ab-b3e0-4bd1-8a5d-8a1a72a75987.7c135cc1-5b81-4ef7-be77-adad018e5e17.7c135cc1-5b81-4ef7-be77-adad018e5e17.bitdep||v1,Value
2022-01-26 07:42:42,2.0,2.0
2022-01-26 07:42:43,2.0,2.0
2022-01-26 07:42:44,2.0,2.0
2022-01-26 07:42:45,2.0,2.0
2022-01-26 07:42:46,2.0,2.0
...,...,...
2022-11-15 11:35:40,0.0,0.0
2022-11-15 11:35:41,0.0,0.0
2022-11-15 11:35:42,0.0,0.0
2022-11-15 11:35:43,0.0,0.0


In [163]:
_pd_real_time

,dw:witsml:log.5a07f5ab-b3e0-4bd1-8a5d-8a1a72a75987.7c135cc1-5b81-4ef7-be77-adad018e5e17.7c135cc1-5b81-4ef7-be77-adad018e5e17.bitdep||v1,Value
2022-01-26 07:42:42,2.0,2.0
2022-01-26 07:42:43,2.0,2.0
2022-01-26 07:42:44,2.0,2.0
2022-01-26 07:42:45,2.0,2.0
2022-01-26 07:42:46,2.0,2.0
...,...,...
2022-11-15 11:35:40,0.0,0.0
2022-11-15 11:35:41,0.0,0.0
2022-11-15 11:35:42,0.0,0.0
2022-11-15 11:35:43,0.0,0.0


In [164]:

_pd_real_time_plot = _pd_real_time
# da['new_index'] = str(da.index)
# da = da.drop(columns=['dw:witsml:log.1ac57fe7-4c90-4bc7-a350-8d17f077bb32.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.bitdep||v1'])
# da.set_index('new_index')

In [165]:
_pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
_pd_real_time_plot['Date'] = _pd_real_time_plot.index
_pd_real_time_plot

,Value,Date
2022-01-26 07:42:42,2.0,2022-01-26 07:42:42
2022-01-26 07:42:43,2.0,2022-01-26 07:42:43
2022-01-26 07:42:44,2.0,2022-01-26 07:42:44
2022-01-26 07:42:45,2.0,2022-01-26 07:42:45
2022-01-26 07:42:46,2.0,2022-01-26 07:42:46
...,...,...
2022-11-15 11:35:40,0.0,2022-11-15 11:35:40
2022-11-15 11:35:41,0.0,2022-11-15 11:35:41
2022-11-15 11:35:42,0.0,2022-11-15 11:35:42
2022-11-15 11:35:43,0.0,2022-11-15 11:35:43


In [166]:
# from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature as sst

# sst.hvplot()

In [167]:
min(_pd_real_time_plot['Date'])
# Timestamp('2022-01-26 07:42:42')

Timestamp('2022-01-26 07:42:42')

In [168]:
max(_pd_real_time_plot['Date'])

Timestamp('2022-11-15 11:35:44')

In [169]:
# _pd_real_time_plot[pd.isnull(_pd_real_time_plot).any(axis=1)]

## plot real time data

In [170]:
# _pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y=['Date'], 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)

## plot with panel

In [171]:
# sr2 = pd.Series(pd.Series(_pd_witsml_flattened['id']).tolist(), index = _pd_witsml_flattened['id'])
# # d = {key: value for key, value in sr2.items()}
# # _sensorids_list = d
# _sensorids_list = pd.Series(_pd_witsml_flattened['id']).tolist()



# # create the Select widget
# _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list, width=200)



# # def _update__sel_sensorid
# # @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
# def _update__sel_sensorid(value):
#     _sel_sensorid = value
#     # do something with _sel_sensorid
#     return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
# pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

#### make plot

In [172]:

# # make plot of real time data
# plot =_pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y='Date', 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)




#### show panel

In [173]:

# # bind the update function to the widget
# _sel_sensorid_widget.param.watch(_update__sel_sensorid, 'value')

# # display the widget and plot side by side
# pn.Row(_sel_sensorid_widget, plot,_sel_sensorid_widget.value).servable()

## trying with plotly

In [174]:
# pn.pane.Perspective(
#     _pd_real_time,columns=['Value']
# ).servable()

## trying to bind properly

In [175]:
# pn.extension(design='material')
 
# csv_file = ("https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv")

# data = pd.read_csv(csv_file, parse_dates=["date"], index_col="date")
 
# data.tail()
 
 
 
# def transform_data(variable, window, sigma):
#     ''' Calculates the rolling average and the outliers '''
#     avg = data[variable].rolling(window=window).mean()
#     residual = data[variable] - avg
# #     std = residual.rolling(window=window).std()
# #     outliers = np.abs(residual) > std * sigma
# #     return avg, avg[outliers]
 
# def create_plot(df):
#     ''' Plots data '''
#     # avg, highlight = transform_data(variable, window, sigma)
#     plot = df.hvplot(height=1000,
#         width=500, title='Real Time',
#         x='Value',
#         y=['Date'], 
#         ylim=(min(df['Date']), max(df['Date']))
#         ).opts(invert_yaxis=True)
#     return plot
#     # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
#     #     color="orange", padding=0.1, legend=False
#     # )
 
 
 
 
# variable_widget = pn.widgets.Select(name="variable", options=list(_pd_real_time_plot.columns))
# window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
# sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)
# test_widget = pn.widgets.Button(name="testing", button_type="primary")
 
 
# # variable_widget.servable()
 
 
 
# # create_plot(data=_pd_real_time_plot, window=20, sigma=10)
 
 
 
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')
 
# def update_indicator(event, clicks):
#     if not event:
#         return
   
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"
 
# # pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)
 
# pn.Column(pn.Row(test_widget, text),indicator)
 
 
 
 
 
 
# bound_plot = pn.bind(create_plot, variable=variable_widget, window=window_widget, sigma=sigma_widget)
 
 
 
 
 
# first_app = pn.Column(variable_widget, window_widget, test_widget, sigma_widget, bound_plot)
# first_app.servable()

In [176]:
# #plot data 
# def create_plot(df, wellbore, sensortype, df_witsml):
#     ''' Plots data '''
#     # avg, highlight = transform_data(variable, window, sigma)
#     # id, df = _make_pd_real_time_plot(_get_sensorid(_pd_witsml_flattened))
#     # df = _new_plot
#     _w_wellbores_rx = _w_wellbores_select(df_witsml).rx()
    
#     plot = df.hvplot(height=1000,
#         width=500, title= 'Real Time data for ' + wellbore + " " + sensortype,
#         x='Value',
#         y=['Date'], 
#         ylim=(min(df['Date']), max(df['Date']))#,
        
#         #widgets={'nameWellbore' : _w_wellbores}
#         # ).opts(invert_yaxis=True)
#     )
    
#     _pn_plot = pn.Row(pn.Column(_w_wellbores_rx ),pn.rx(plot))
#     return plot
#     # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
#     #     color="orange", padding=0.1, legend=False
#     # )





# # make datasets
# def _make_pd_real_time_plot(sensorid) :
#     """
#     make pd of real time dat to be plotted for given sensorid
#     Args: sensorid - the id of the sensor
#     Returns: the id column and the pandas to be plotted
#     """
#     _pd_real_time_plot = _make_pd_real_time(sensorid)
#     _realtimedata_id = _pd_real_time_plot.columns
#     _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
#     _pd_real_time_plot['Date'] = _pd_real_time_plot.index
#     return _realtimedata_id, _pd_real_time_plot

# def _make_pd_real_time(sensorid):
#     """make pd of real time data for given sensorid"""
#     _pd_real_time = _get_df_real_time(sensorid).to_pandas()
#     # make Value columsn or easier filtering
#     _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
#     return _pd_real_time

# def _get_df_real_time(sensorid):
#     """make cognite client of real time data for given sensorid"""
#     _df_real_time = client.time_series.data.retrieve(
#         id=sensorid, # TODO make function to choose
#         start=MIN_TIMESTAMP_MS,
#         end=MAX_TIMESTAMP_MS + 1) 
#     return _df_real_time   

# # get sensor id
# def _get_sensorid(df):
#     """
#         Args: df on the format of _pd_witsml_flattened
#         Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
#     """
#     # _chosen_wellbore = str(_w_wellbores_select(df).value)
#     _chosen_wellbore = _w_wellbores_select(df).value
#     _chosen_sensortype= str(_w_sensortypes_select(df).value)
#     # _chosen_sensortype= _chosen_sensortype.value
#     _sensor_row = df[(df['nameWellbore'] == _chosen_wellbore) & (df['uid'] == _chosen_sensortype)]
#     _sensorid = int(_sensor_row['id'].iloc[0])
#     _sensortype = _sensor_row['uid'].iloc[0]
#     _wellbore = _sensor_row['nameWellbore'].iloc[0]
#     return _sensorid, _wellbore, _sensortype




# # make list for the widgets

# # def _make_sensorids_list(df):
# #     sr2 = pd.Series(pd.Series(df['id']).tolist(), index = df['id'])
# #     d = {key: value for key, value in sr2.items()}
# #     _sensorids_list = d
# #     return _sensorids_list


# def _make_wellbores_list(df):
#     # sr2 = pd.Series(pd.Series(df['nameWellbore']).unique().tolist(), index = df['id'])
#     _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
#     # d = {key: value for key, value in sr2.items()}
#     # _sensorids_list = d
#     return _wellbores_list

# def _make_sensortypes_list(df):
#     # sr2 = pd.Series(pd.Series(df['uid']).unique().tolist(), index = df['id']) # TODO check that uid is same as name aka sensortype
#     _sensortypes_list = pd.Series(df['uid']).unique().tolist()
#     # d = {key: value for key, value in sr2.items()}
#     # _sensorids_list = d
#     return _sensortypes_list




# ## make widgets
# def _w_wellbores_select(df):
#     """
#     Args: df on the format of _pd_witsml_flattened
#     """
#     _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_make_wellbores_list(df))
#     return _sel_sensorid_widget

# def _w_sensortypes_select(df):
#     """
#     Args: df on the format of _pd_witsml_flattened
#     """
#     _sel_sensortype_widget = pn.widgets.Select(name='Select', options=_make_sensortypes_list(df))
#     return _sel_sensortype_widget

# # get widget values
# def _get_wellbore():
#     return _w_wellbores_select().value

# def _get_sensortype():
#     return _w_sensortypes_select().value

# # @pn.depends(_w_wellbores_select(_pd_witsml_flattened))
# # def _new_plot():
#     # return _w_wellbores_select(_pd_witsml_flattened).value

# def b(event):
#     print("hellp")



# # define the update function
# def update_plot(event):
#     sensorid = _get_sensorid(_pd_witsml_flattened)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # _w_wellbores_select(df)



# # _w_wellbores_select =  select(_pd_witsml_flattened)
# # _w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# # define the update function
# def update_plot(wellbore):
#     sensorid = _get_sensorid(_pd_witsml_flattened, wellbore)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, wellbore, _w_sensortypes_select.value, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # bind the update function to the widget values
# _w_wellbores_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

# _w_sensortypes_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

# # display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()



# # create the Select widgets
# # _w_wellbores_select_widget = _w_wellbores_select(_pd_witsml_flattened)
# # _w_sensortypes_select_widget = pn.widgets.Select(name='Select Sensor Type', options=_make_sensortypes_list(_pd_witsml_flattened))

# # bind the update function to the widget values
# #_w_wellbores_select(_pd_witsml_flattened).param.watch(update_plot, 'value')
# # pn.bind(update_plot, pn.widgets.Select(name='Select Wellbore', options=_make_wellbores_list(_pd_witsml_flattened)))
# # _w_sensortypes_select.param.watch(update_plot, 'value')

# # \pn.bind(update_plot, event='new')
# # pn.bind(update_plot, event='new')

# # pn.bind(update_plot, _w_wellbores_select_widget,  watch=True)

# # _w_wellbores_select.param.value.bind(update_plot)
# # _w_sensortypes_select.param.value.bind(update_plot)

In [245]:
#### REWQRITE PROPER NEW ONES


# make lists to use in widgets
def _make_wellbores_list(df):
    _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
    return _wellbores_list

def _make_sensortypes_list(df):
    _sensortypes_list = pd.Series(df['uid']).unique().tolist()
    return _sensortypes_list




## make widgets
def _w_wellbores_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _wellbore = pn.widgets.Select(name='Select wellbore', options=_make_wellbores_list(df))
    return _wellbore

def _w_sensortypes_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sensor_type = pn.widgets.Select(name='Select sensor type', options=_make_sensortypes_list(df))
    return _sensor_type


# Set variables
_w_sensortype = _w_sensortypes_select(_pd_witsml_flattened)
_w_wellbore = _w_wellbores_select(_pd_witsml_flattened)







#plot data 
def create_plot(_interactive, test="false"):#, wellbore, sensortype, df_witsml):
    ''' Plots data '''    
    # _sensortype = _w_sensortype.param.get_param_values().__getitem__(24)[1]
    print("below is from create_plot")
    _sensortype = _w_sensortype
    _wellbore = _w_wellbore
    # print(_sensortype)
    line_plot = _interactive.hvplot(height=1000,

        width=500, title= 'Real Time data for ' + " ",# + str(dict(_w_sensortype.param.values().items())['value']),# + _wellbore,
        x='Value',
        xlabel = _sensortype,
        # hover_cols=['Description'],
        # y=['Date'],
        y=['Date_Seconds'],

        autorange="y"
        # value_label = 'Date'
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
    constant_x = 10
    # x=pd.Series([constant_x] * num_rows)
    # scatter_plot = _interactive.hvplot(y=['DescriptionDate'],hover_cols=['Description'], kind='scatter')
    # combined_plot  =plot * scatter_plot

        ### THIS ONE - edit with pd.Timestamp() instead
    # Define the coordinates of the rectangles
    rectangles = [
        {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
        {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
        {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
    ]


    # Convert the y values to Unix timestamps
    for rectangle in rectangles:
        rectangle['ystart'] = rectangle['ystart'].timestamp()
        rectangle['yend'] = rectangle['yend'].timestamp()


    # setting global values for styling graps
    HEIGHT = 500
    WIDTH = 500
    
    # Create a list to store the polygons and text labels
    plots = []

    # Add the rectangles to the plot
    for rectangle in rectangles:
        # Define the vertices of the rectangle
        vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                    (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                    (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
        

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)
    # Combine all the plots into a single plot
    combined_plot = line_plot + hv.Overlay(plots)
    # Display the plot
    # combined_plot = hv.Layout([line_plot, hv.Overlay(plots)]).cols(2).opts(shared_axes=True)

    return (combined_plot).opts(shared_axes=True)

# Define a function to update the labels
def update_labels( plot, well, sensor, y_label= 'Date'):
    plot.opts(xlabel = sensor, title=f"Real time data for {well} - {sensor}")








# make datasets
def _make_pd_real_time_plot(sensorid) :
    """
    make pd of real time dat to be plotted for given sensorid
    Args: sensorid - the id of the sensor
    Returns: the id column and the pandas to be plotted
    """
    _pd_real_time_plot = _make_pd_real_time(sensorid)
    _realtimedata_id = _pd_real_time_plot.columns
    _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
    _pd_real_time_plot['Date'] = pd.to_datetime(_pd_real_time_plot.index)
    _pd_real_time_plot['Date_Seconds'] = _pd_real_time_plot['Date'].apply(lambda x: x.timestamp())
    # _pd_real_time_plot['Date'] =_pd_real_time_plot.index
    return _realtimedata_id, _pd_real_time_plot

def _make_pd_real_time(sensorid):
    """make pd of real time data for given sensorid"""
    _pd_real_time = _get_df_real_time(sensorid).to_pandas()
    # make Value columsn or easier filtering
    _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
    return _pd_real_time

def _get_df_real_time(sensorid):
    """make cognite client of real time data for given sensorid"""
    _df_real_time = client.time_series.data.retrieve(
        external_id= None,
        id=sensorid, # TODO make function to choose
        start=MIN_TIMESTAMP_MS, # TODO change to acutal vlaues,min(df['Date']) 
        end=MAX_TIMESTAMP_MS + 1) 
    return _df_real_time   

# get sensor id
def _get_sensorid(df, wellbore, sensortype):
    """
        Args: df on the format of _pd_witsml_flattened
        Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
    """
    _sensor_row = df[(df['nameWellbore'] == wellbore) & (df['uid'] == sensortype)]
    _sensorid = _sensor_row['id'].iloc[0]
    return _sensorid





In [178]:
_w_wellbores_select(_pd_witsml_flattened).value

'NO 16/2-D-2 T4'

In [179]:
# display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [180]:

# sensorid, wellbore, sensor =  _get_sensorid(_pd_witsml_flattened)
# id, data = _make_pd_real_time_plot(sensorid)


# # #btn = pn.widgets.Button(name = "click me", button_type = 'primary')

# plot = create_plot(data, wellbore, sensor, _pd_witsml_flattened)
# # plot.param.trigger('draw')
# plot

In [181]:
# import panel as pn
# import pandas as pd
# import xarray as xr
# import hvplot.xarray

# # create the Select widgets
# _w_wellbores_select = pn.widgets.Select(name='Select Wellbore',  options=_make_wellbores_list(_pd_witsml_flattened))
# _w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# # create the DataFrame and plot
# _pd_real_time_plot = pd.DataFrame({
#     'Value': [1, 2, 3, 4, 5],
#     'Date': pd.date_range('2022-01-01', periods=5, freq='D')
# })

# plot = _pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y=['Date'], 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)

# # define the update function
# def update_plot(event):
#     sensorid = _get_sensorid(_pd_witsml_flattened)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # bind the update function to the widget values
# _w_wellbores_select.param.watch(update_plot, 'value')
# _w_sensortypes_select.param.watch(update_plot, 'value')

# # display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [182]:
# import pandas as pd
# import numpy as np
# import hvplot.pandas
# import panel as pn

# # create dataset d1
# d1 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10),
#     'y2': np.random.rand(10)
# })

# # create dataset d2 with different column names
# d2 = pd.DataFrame({
#     'x': np.arange(10),
#     'z1': np.random.rand(10),
#     'z2': np.random.rand(10)
# })

# # plot d1 using hvplot
# plot = d1.hvplot.line(x='x', y=['y1', 'y2'])

# # define the update function
# def update_plot(event):
#     #global plot
#     newplot = d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")
#     plot = newplot
#     plot

# # create the button widget
# button = pn.widgets.Button(name='Update Plot')


# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')
 
# def update_indicator(event, clicks):
#     if not event:
#         return
   
#     indicator.value = not indicator.value

#     test = pn.Column(button, (d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")), indicator, text).servable()
#     test.servable
#     text.value = f"you have clicked me {clicks} times"
#     plot.title = "hjhkj"
 
# # pn.bind(update_indicator, button, button.param.clicks, watch=True)

# # pn.bind(update_plot, button, watch=True)

# # bind the update function to the button click event
# # button.on_click(update_plot)

# # display the button and plot using Panel
# # pn.Column(button, plot, indicator, text).servable()
# hvplot.bind(update_plot, button).interactive().hvplot.line(title="d2")


In [183]:
# import hvplot.pandas  # noqa
# import pandas as pd
# import panel as pn

# pn.extension()


# def function_to_generate_dataframe(x):
#     # x only used to trigger the function
#     df =  pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10),
#     'y2': np.random.rand(10)
# })
#     # df["C"] = df["C"].round()
#     return df


# widget = pn.widgets.IntSlider(end=100)
# hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()


In [184]:
# import hvplot.pandas
# import pandas as pd
# import panel as pn
# import numpy as np

# pn.extension()

# # create dataset d1
# d1 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10)
# })


# # create dataset d2 with different column names
# d2 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10)
#     })

# def function_to_generate_dataframe(x):
#     # x only used to trigger the function
#     global d1, d2
#     if x == 'd1':
#         df = d1
#     else:
#         df = d2
    
#     return df.hvplot.line(x='x', title=x)

# widget = pn.widgets.Select(options=['d1', 'd2'])
# hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()

In [185]:
#WORKS


# # create dataset d1
# d1 = _pd_real_time_plot


# # create dataset d2 with different column names
# d2 = _pd_real_time_plot

# def function_to_generate_dataframe2(x):
#     # x only used to trigger the function
#     global d1, d2
#     if x == 'd1':
#         df = d1
#     else:
#         df = d2
    
#     return df.hvplot.line(x='Value', title=x)

# widget2 = pn.widgets.Select(options=['d1', 'd2'])
# p = hvplot.bind(function_to_generate_dataframe2, widget2).interactive()
# p.hvplot().line()


In [186]:
# ### WORKs as of 20.06 
# #tesing with new code and proper WORRKSSS YESSS
# # import hvplot.pandas
# from bokeh.models import DatetimeTickFormatter

# def master(wellbore, sensortype):
#     # print(wellbore, sensortype)
#     # print(type(sensorid))
#     # print(sensorid)
#     # s= int(sensorid)
#     # global num_rows

#     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
#     plot_opts = dict(height=500, width=500, xlabel =dict(_w_sensortype.param.values().items())['value'], yformatter=DatetimeTickFormatter(days="%m/%d",
#                                             hours="%H",
#                                             minutes="%H:%M") )#'%Y-%m-%d %H:%M:%S')
    
#     # param.depends(_w_sensortype)(scatter_plot.plot)
#     print("finished in master")
    


#     return data#, plot


# def _combine_datasets(wellbore,sensortype):
#     sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)

#     id, data = _make_pd_real_time_plot(int(sensorid))

#     # test= pd.DataFrame({'DescriptionDateStart':pd.to_datetime(['2022-11-24 05:27:00', '2022-11-25 05:27:10', '2022-11-27 05:27:20', '2022-11-29 05:30:10']),
#     # 'DescriptionDateEnd':pd.to_datetime(['2022-11-25 05:27:00', '2022-11-26 05:27:10', '2022-11-28 05:27:20', '2022-12-01 05:30:10']),

#     # 'x_value': pd.Series([50000000000000000.00, 50000000000000000.00, 50000000000000000.00,50000000000000000.00]),
#     # 'Description': pd.Series(["yo","hello","everyone","here"]) })


#     # combined_df = pd.concat([data, test], axis=0, ignore_index=True)
#     # data = combined_df
#     num_rows = len(data)
#     min_d = min(data['Date'])
#     max_d = max(data['Date'])

#     # data['test_date'] = data['Date'].apply(lambda x: x.timestamp())
#     # data['proper_date'] = pd.to_datetime(data['Date'])


#     return data, num_rows

# # def _get_num_rows(wellbore,sensortype):
# #     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
# #     return num_rows







# # # get the current value of the try_sensor widget
# # def callback_sensortype(event):
# #     sensortype = event
# #     return sensortype

# # # bind the callback function to the _w_sensortypes_select widget
# # _w_sensortype.param.watch(callback_sensortype, 'value')
# # # try_sensor

# # def get_chosen_sensortype():
# #     # get the current value of the try_sensor widget using the callback_sensortype function
# #     sensortype = callback_sensortype( _w_sensortype.value)
# #     print("sensortype")
# #     print(sensortype)
# #     return sensortype


# def update_title():
#     create_plot().title="new"
    

# def callback_sensortype(event):
#     # get the current value of the _w_sensortype widget
#     print(type(event))
#     sensortype = event.new
#     print(type(sensortype))
#     # plot.title="new"
#     # _w_sensortype.observe(update_title, 'value')
#     # _w_sensortype.select(update_title, 'value')
#     # print(type(_w_sensortype.__getattribute__('value')))
#     # print(_w_sensortype.__getattribute__('value'))
#     return sensortype

# # bind the callback function to the _w_sensortype widget
# _w_sensortype.param.watch(callback_sensortype, 'value')




# def _make_interactive():
#     _interactive = hvplot.bind\
#     (master,_w_wellbore, _w_sensortype)\
#     .interactive()
#     # Call the function to update the labels
#     print("finished")

#     return _interactive, dict(_w_sensortype.param.values().items())['value']
# # def master():
# #     function_to_generate_dataframe(_w_wellbore, _w_sensortype) 


# # def _make_interactive():
# #     _interactive = hvplot.bind\
# #     (master)\
# #     .interactive()
# #     print("finished")
# #     return _interactive#, _w_sensortype.param.get_param_values().__getitem__(24)[1]

# _interactive, s = _make_interactive()

# # get_chosen_sensortype()
# # _w_sensortype

# # callback_sensortype(_w_sensortype)

# # _make_interactive().head(5)

# # plot = create_plot(_make_interactive()[0],_make_interactive()[1])
# # plot

finished in master
finished


In [216]:
### WORKs as of 20.06 - COPY for test
#tesing with new code and proper WORRKSSS YESSS
# import hvplot.pandas
from bokeh.models import DatetimeTickFormatter

def master(wellbore, sensortype):
    # print(wellbore, sensortype)
    # print(type(sensorid))
    # print(sensorid)
    # s= int(sensorid)
    # global num_rows

    data = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
    plot_opts = dict(height=500, width=500, xlabel =dict(_w_sensortype.param.values().items())['value'], yformatter=DatetimeTickFormatter(days="%m/%d",
                                            hours="%H",
                                            minutes="%H:%M") )#'%Y-%m-%d %H:%M:%S')
    
    # param.depends(_w_sensortype)(scatter_plot.plot)
    print(str(_w_sensortype))
    print(dict(_w_sensortype.param.values().items())['value'])
    print("finished in master")
    # create_plot(data)
    # plot = create_plot(data)

    return data#, plot


def _combine_datasets(wellbore,sensortype):
    sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)

    id, data = _make_pd_real_time_plot(int(sensorid))

    # test= pd.DataFrame({'DescriptionDateStart':pd.to_datetime(['2022-11-24 05:27:00', '2022-11-25 05:27:10', '2022-11-27 05:27:20', '2022-11-29 05:30:10']),
    # 'DescriptionDateEnd':pd.to_datetime(['2022-11-25 05:27:00', '2022-11-26 05:27:10', '2022-11-28 05:27:20', '2022-12-01 05:30:10']),

    # 'x_value': pd.Series([50000000000000000.00, 50000000000000000.00, 50000000000000000.00,50000000000000000.00]),
    # 'Description': pd.Series(["yo","hello","everyone","here"]) })


    # combined_df = pd.concat([data, test], axis=0, ignore_index=True)
    # data = combined_df
    # num_rows = len(data)
    # min_d = min(data['Date'])
    # max_d = max(data['Date'])

    # data['proper_date'] = pd.to_datetime(data['Date'])


    return data#, num_rows

# def _get_num_rows(wellbore,sensortype):
#     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
#     return num_rows







# # get the current value of the try_sensor widget
# def callback_sensortype(event):
#     sensortype = event
#     return sensortype

# # bind the callback function to the _w_sensortypes_select widget
# _w_sensortype.param.watch(callback_sensortype, 'value')
# # try_sensor

# def get_chosen_sensortype():
#     # get the current value of the try_sensor widget using the callback_sensortype function
#     sensortype = callback_sensortype( _w_sensortype.value)
#     print("sensortype")
#     print(sensortype)
#     return sensortype


def update_title():
    create_plot().title="new"
    

def callback_sensortype(event):
    # get the current value of the _w_sensortype widget
    print(type(event))
    sensortype = event.new
    print(type(sensortype))
    # plot.title="new"
    # _w_sensortype.observe(update_title, 'value')
    # _w_sensortype.select(update_title, 'value')
    # print(type(_w_sensortype.__getattribute__('value')))
    # print(_w_sensortype.__getattribute__('value'))
    return sensortype

# bind the callback function to the _w_sensortype widget
_w_sensortype.param.watch(callback_sensortype, 'value')




def _make_interactive():
    _interactive = hvplot.bind\
    (master,_w_wellbore, _w_sensortype)\
    .interactive()
    # Call the function to update the labels
    print("finished")



    return _interactive#, dict(_w_sensortype.param.values().items())['value']
# def master():
#     function_to_generate_dataframe(_w_wellbore, _w_sensortype) 


# def _make_interactive():
#     _interactive = hvplot.bind\
#     (master)\
#     .interactive()
#     print("finished")
#     return _interactive#, _w_sensortype.param.get_param_values().__getitem__(24)[1]

_interactive = _make_interactive()

# get_chosen_sensortype()
# _w_sensortype

# callback_sensortype(_w_sensortype)

# _make_interactive().head(5)

# plot = create_plot(_make_interactive()[0],_make_interactive()[1])
# plot

Select(options=['CEMPUMPPR', ...], sizing_mode='stretch_width', value='CEMPUMPPR')
CEMPUMPPR
finished in master
finished


In [195]:
print(str(_w_sensortype))
print(dict(_w_sensortype.param.values().items())['value'])

Select(options=['CEMPUMPPR', ...], sizing_mode='stretch_width', value='RH_YP_COR50')
RH_YP_COR50


In [248]:
create_plot(_interactive)

below is from create_plot
[(2000000000000.0, 1671946140.0), (22000000000000.0, 1671946140.0), (22000000000000.0, 1671600660.0), (2000000000000.0, 1671600660.0)]


BokehModel(combine_events=True, render_bundle={'docs_json': {'fccd742a-4f9c-428f-89a2-cd4f2ee0d4d5': {'version…

In [247]:


# Create some sample data
df1 = pd.DataFrame({'x': np.arange(10), 'y': np.random.rand(10)})
df2 = pd.DataFrame({'x': np.arange(10), 'y': np.random.rand(10)})

# Create two plots using hvplot
plot1 = df1.hvplot.line(x='x', y='y')
plot2 = df2.hvplot.line(x='x', y='y')

# Combine the plots into a layout with shared y-axis
layout = hv.Layout([plot1, plot2]).cols(2).opts(shared_axes=True)

# Display the layout
layout

:Layout
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [ ]:

def _make_interactive_create_plot():
    data =  master(_w_wellbore, _w_sensortype)
    _int_plot = hvplot.bind\
    (create_plot, data)\
    .interactive()
    # Call the function to update the labels
    print("finished")
_make_interactive_create_plot()

IndexError: single positional indexer is out-of-bounds

In [ ]:
hv.help(hv.Polygons)

Polygons

Online example: http://holoviews.org/reference/elements/bokeh/Polygons.html

-------------
Style Options
-------------

	alpha, cmap, color, fill_alpha, fill_color, hover_alpha, hover_color, hover_fill_alpha, hover_fill_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_join, hover_line_width, line_alpha, line_cap, line_color, line_dash, line_join, line_width, muted, muted_alpha, muted_color, muted_fill_alpha, muted_fill_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_fill_alpha, nonselection_fill_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_join, nonselection_line_width, selection_alpha, selection_color, selection_fill_alpha, selection_fill_color, selection_line_alpha, selection_line_cap, selection_line_color, selection_line_dash, selection_line_join

In [ ]:

plot = create_plot(_interactive)
plot
# # Create two dataframes
# df1 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
# df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [7, 8, 9], 'desc': ["yo", "hello", "there"]})

# # Create two plots
# plot1 = df1.hvplot.line(x='x', y='y')
# plot2 = df2.hvplot.scatter(x='x', y='y', hover_cols = ['desc'],color= "#f16a6f")

# # Overlay the plots
# overlayed_plot = plot1 * plot2

# # Display the overlayed plot
# overlayed_plot

below is from create_plot


TypeError: can only concatenate str (not "Select") to str

In [ ]:
update_labels(plot, "well", s)


In [ ]:
dict(_w_sensortype.param.values().items())['value']

'CEMPUMPPR'

In [ ]:
### THIS ONE - edit with pd.Timestamp() instead
# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]


# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['ystart'] = rectangle['ystart'].timestamp()
    rectangle['yend'] = rectangle['yend'].timestamp()


# setting global values for styling graps
HEIGHT = 500
WIDTH = 500
# Create a line plot
line_plot = _interactive.hvplot(height=HEIGHT,
    width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y=['Date']# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
    

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot + hv.Overlay(plots)

# Display the plot
combined_plot

#TODO make plot object and updatexlable attribute?



[(2000000000000.0, 1669267620.0), (22000000000000.0, 1669267620.0), (22000000000000.0, 1669181220.0), (2000000000000.0, 1669181220.0)]
[(2000000000000.0, 1669699500.0), (22000000000000.0, 1669699500.0), (22000000000000.0, 1669440180.0), (2000000000000.0, 1669440180.0)]
[(2000000000000.0, 1671946140.0), (22000000000000.0, 1671946140.0), (22000000000000.0, 1671600660.0), (2000000000000.0, 1671600660.0)]


BokehModel(combine_events=True, render_bundle={'docs_json': {'2f7d19a6-9f6e-4bc1-9898-04c6bdcd88bf': {'version…

In [ ]:
import pandas as pd
import holoviews as hv

# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]

# Set the plot options
plot_opts = dict(height=500, width=500, xaxis=None, yformatter='%Y-%m-%d %H:%M:%S')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart'].timestamp()), 
                (rectangle['x'] + rectangle['width'], rectangle['ystart'].timestamp()),
                (rectangle['x'] + rectangle['width'], rectangle['yend'].timestamp()),
                (rectangle['x'], rectangle['yend'].timestamp())]

    # Set the fill color of the rectangle based on the label
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).options(alpha=0.5, color=fill_color)

    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['yend'] - rectangle['ystart']) / 2, rectangle['label']).options(color='black', xaxis=None)

    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = hv.Overlay(plots).options(**plot_opts)

# Display the plot
combined_plot

:Overlay
   .Polygons.I   :Polygons   [x,y]
   .Text.I       :Text   [x,y]
   .Polygons.II  :Polygons   [x,y]
   .Text.II      :Text   [x,y]
   .Polygons.III :Polygons   [x,y]
   .Text.III     :Text   [x,y]

In [ ]:
combined_plot

:Overlay
   .Polygons.I   :Polygons   [x,y]
   .Text.I       :Text   [x,y]
   .Polygons.II  :Polygons   [x,y]
   .Text.II      :Text   [x,y]
   .Polygons.III :Polygons   [x,y]
   .Text.III     :Text   [x,y]

In [ ]:
### THIS ONE COPY - works for timestamp() 
#  # Define the coordinates of the rectangles

# from bokeh.plotting import figure, output_file, save, Show
# from bokeh.models import DatetimeTickFormatter
# import datetime as dt
# import datetime as dt

rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-10 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]


# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['ystart'] = rectangle['ystart'].timestamp()
    rectangle['yend'] = rectangle['yend'].timestamp()


# setting global values for styling graps
# HEIGHT = 500
# WIDTH = 500
# Create a line plot
line_plot = _interactive.hvplot(height=HEIGHT,
    width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y=['Date']# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []



# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
    

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot.options(**plot_opts) + hv.Overlay(plots)#.options(**plot_opts)

# Display the plot
combined_plot




[(2000000000000.0, 1669267620.0), (22000000000000.0, 1669267620.0), (22000000000000.0, 1669181220.0), (2000000000000.0, 1669181220.0)]
[(2000000000000.0, 1669699500.0), (22000000000000.0, 1669699500.0), (22000000000000.0, 1669440180.0), (2000000000000.0, 1669440180.0)]
[(2000000000000.0, 1670650140.0), (22000000000000.0, 1670650140.0), (22000000000000.0, 1669008660.0), (2000000000000.0, 1669008660.0)]


BokehModel(combine_events=True, render_bundle={'docs_json': {'6c525d3e-c79d-458c-b0ae-92b95e56fb59': {'version…

In [ ]:
_interactive = hvplot.bind\
    (function_to_generate_dataframe, _w_wellbores_select(_pd_witsml_flattened), _w_sensortype, lambda param1, param2: _w_another_function(param1, param2)) \
    .interactive()

NameError: name 'function_to_generate_dataframe' is not defined

In [ ]:
bound_functions

TypeError: unhashable type: 'list'

<function param.reactive.bind.<locals>.wrapped(*wargs, **wkwargs)>

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]

# Convert the rectangles to a Pandas DataFrame
df = pd.DataFrame(rectangles)

# Create the fill_color column based on the label of each rectangle
# df['fill_color'] = ['red' if '- D' in label else 'green' if '- U' in label else 'orange' for label in df['label']]

# Define the plot options
plot_opts = dict(height=500, width=500, xaxis=None, yformatter='%Y-%m-%d %H:%M:%S')

# Create the area plot
area_plot = df.hvplot.area(x='x', alpha=0.5, line_color=None, hover_alpha=1.0, hover_line_color='black').options(**plot_opts)

# Display the plot
area_plot

:Area   [x]   (width)

In [ ]:
area_plot

:NdOverlay   [Variable]
   :Area   [index]   (value,Baseline)

In [ ]:
combined_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'53dbbe85-e8cd-4b42-b343-66a0a1020338': {'version…

In [ ]:


# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': '2022-11-24 05:27:00', 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': '2022-11-24 05:25:00', 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': '2022-11-24 05:29:00', 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}

]
# df['Date'] = df['Date'].apply(lambda x: x.timestamp())

# Create a line plot
line_plot = _interactive.hvplot(height=500,
    width=250, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y='Date'# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

TypeError: can only concatenate str (not "int") to str

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': pd.Timestamp('2022-11-24 05:27:00'), 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': pd.Timestamp('2022-11-24 05:25:00'), 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': pd.Timestamp('2022-11-24 05:29:00'), 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}
]

# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['y'] = rectangle['y'].timestamp()

# Create a line plot
df = pd.DataFrame({'Value': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'Date': pd.date_range('2022-11-24 05:25:00', periods=10, freq='T')})
line_plot = df.hvplot.line(x='Value', y='Date')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Convert the y-coordinates to float values
    vertices = [(x, float(y)) for x, y in vertices]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    # label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    # plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': pd.Timestamp('2022-11-24 05:27:00'), 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': pd.Timestamp('2022-11-24 05:25:00'), 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': pd.Timestamp('2022-11-24 05:29:00'), 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}
]

# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['y'] = rectangle['y'].timestamp()

# Create a line plot with Unix timestamp y values
df = pd.DataFrame({'Value': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'Date': pd.date_range('2022-11-24 05:25:00', periods=10, freq='T')})
df['Date'] = df['Date'].apply(lambda x: x.timestamp())
line_plot = df.hvplot.line(x='Value', y='Date')


# line_plot = _interactive.hvplot(height=500,
#     width=250, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
#     x='Value',
#     # xlabel = _sensortype,
#     # hover_cols=['Description'],
#     y=['Date']# ,
#     # value_label = 'Date'
#     # ,ylim=(min(df['Date']), max(df['Date']))#,
# )



# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Convert the y-coordinates to float values
    vertices = [(x, float(y)) for x, y in vertices]
    print(vertices)
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
df.head(10)

In [ ]:
# Define the coordinates of the rectangle
x1, y1 = 1, 4
x2, y2 = 3, 6

# Create a dataframe with the coordinates of the rectangle
df = pd.DataFrame({'x': [x1, x2, x2, x1], 'y': [y1, y1, y2, y2]})

# Create an area plot for the rectangle
plot = df.hvplot.area(x='x', y='y', alpha=0.5)

# Display the plot
plot

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import matplotlib.dates as mdates

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'label': ['Rectangle 1', 'Rectangle 2']
    
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
# df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the y2 coordinate of each rectangle
# df['y2'] = df['starttime']

# Create a new column with the y1 coordinate of each rectangle
# df['y1'] = df['y2'] - df['duration']

# Create an area plot for the rectangles with inverted axes
plot = df.hvplot.area(x='starttime', y='starttime', x2='endtime', y2='endtime', alpha=0.5, invert_axes=True)

# Convert the datetime objects to numeric values
x_values = mdates.date2num(df['starttime'].to_numpy())
y_values = mdates.date2num(df['starttime'].to_numpy())

# Add text labels to the plot
labels = hv.Text('2022-11-24 05:27:00', '2022-11-24 05:30:10', "test")#.opts(invert_axes=True, color='black')
hv.help(hv.Text)

In [ ]:
%pip install mpld3

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': 4, 'width': 2, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': 5, 'width': 3, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': 3, 'width': 1, 'height': 3, 'label': 'Rectangle 3'}
]

# Create a line plot
df = pd.DataFrame({'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'y': [3, 4, 5, 6, 5, 4, 3, 2, 1, 2]})
line_plot = df.hvplot.line(x='x', y='y')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
hv.help(hv.Text)

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'y1': [y1, y1],
    'y2': [60, 60]
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the x2 coordinate of each rectangle
df['x2'] = df['starttime']

# Create a new column with the x1 coordinate of each rectangle
df['x1'] = df['x2'] - df['duration']

# Create an area plot for the rectangles
plot = df.hvplot.area(x=['y1', 'y2'], y='starttime', y2='endtime', alpha=0.5)

# Display the plot
plot

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'y1': [y1, y1],
    'y2': [60, 60]
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the y2 coordinate of each rectangle
df['y2'] = df['starttime']

# Create a new column with the y1 coordinate of each rectangle
df['y1'] = df['y2'] - df['duration']

# Create an area plot for the rectangles with inverted axes
plot = df.hvplot.area(x='y1', y='starttime', x2='y2', y2='endtime', alpha=0.5)#.opts(inverted=True)

# Display the plot
plot

In [ ]:
_interactive.head(5)

In [ ]:
combined_plot

In [ ]:
pn.Row(plot, plot.Scatter)

In [ ]:
print(plot.count())

In [ ]:
print(len(i.result()))


In [ ]:
plot.Scatter

In [ ]:
_interactive, s =_make_interactive()
# _interactive.head(20)

plot = _interactive.hvplot(height=1000,
        width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
        x='Value',
        # xlabel = _sensortype,
        y=['Date'] 
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
scatter_plot = _interactive.hvplot(y=['DescriptionDate'], hover_cols=['Description'], kind='scatter')
combined_plot  =plot * scatter_plot
# combined_plot

In [ ]:
# combined_plot
_interactive.hvplot(y=['DescriptionDate'], hover_cols=['Description'], kind='scatter')

In [ ]:
# _interactive[!_interactive.isna()]
             
_interactive[_interactive['Date'].notna()].head(5)


In [ ]:
import random
import string


# Create a range of timestamps
timestamps = pd.date_range(start='2022-01-01', end='2022-01-05', freq='H')



# Plot the DataFrame with data points for the 'description' column
plot_test = _interactive.hvplot(height=1000,
        width=500, title= 'Real Time data for ' ,
        x='Value',
        y=['Date'] 
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
# Display the plot
plot_test

In [ ]:
plot

In [ ]:
# new_interactive_obj = _make_interactive()[0].observe(create_plot, _make_interactive()[0],_make_interactive()[1])
_interactive.head(5)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the function that will be used in the interactive object
def my_function(x, y):
    return x + y

# Create the widgets for the interactive object
x_widget = widgets.IntSlider(min=0, max=10, step=1, value=5, description='x:')
y_widget = widgets.IntSlider(min=0, max=10, step=1, value=5, description='y:')

# Create the interactive object using the function and widgets
interactive_obj = widgets.interactive(my_function, x=x_widget, y=y_widget)

# Define a function to update the title of the interactive object
def update_title(change):
    interactive_obj.title = f"x={x_widget.value}, y={y_widget.value}, result={interactive_obj.result}"

# Watch for changes in the value of the x_widget and y_widget, and update the title of the interactive object
x_widget.observe(update_title, 'value')
y_widget.observe(update_title,   'value')

# Display the interactive object
display(interactive_obj)

In [ ]:
### TEST MAKE CLASS


class Interactive(param.parameterized):
    # import hvplot.pandas
    def function_to_generate_dataframe(wellbore, sensortype):
        print(wellbore, sensortype)
        sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)
        print(type(sensorid))
        print(sensorid)
        s= int(sensorid)
        id, data = _make_pd_real_time_plot(s)
        min_d = min(data['Date'])
        max_d = max(data['Date'])
        return data

    _w_sensortype = _w_sensortypes_select(_pd_witsml_flattened)


    # # get the current value of the try_sensor widget
    # def callback_sensortype(event):
    #     sensortype = event
    #     return sensortype

    # # bind the callback function to the _w_sensortypes_select widget
    # _w_sensortype.param.watch(callback_sensortype, 'value')
    # # try_sensor

    # def get_chosen_sensortype():
    #     # get the current value of the try_sensor widget using the callback_sensortype function
    #     sensortype = callback_sensortype( _w_sensortype.value)
    #     print("sensortype")
    #     print(sensortype)
    #     return sensortype
    def callback_sensortype(event):
        # get the current value of the _w_sensortype widget
        print(type(event))
        sensortype = event.new
        print(type(sensortype))
        # print(type(_w_sensortype.__getattribute__('value')))
        # print(_w_sensortype.__getattribute__('value'))
        return sensortype

    # bind the callback function to the _w_sensortype widget
    _w_sensortype.param.watch(callback_sensortype, 'value')



    def _make_interactive():
        _interactive = hvplot.bind\
        (function_to_generate_dataframe, _w_wellbores_select(_pd_witsml_flattened), _w_sensortype) \
        .interactive()
        return _interactive, _w_sensortype.param.get_param_values().__getitem__(24)[1]


    # get_chosen_sensortype()
    # _w_sensortype

    # callback_sensortype(_w_sensortype)

    # _make_interactive().head(5)


In [ ]:
callback_sensortype(_w_sensortype)

In [ ]:
_w_sensortype

In [ ]:
widget = _w_sensortype.param

value = widget.get_param_values().__getitem__(24)[1]
value
# values.__dict__()
# value = widget.value()

In [ ]:
_make_interactive().head(5)

In [ ]:
try_sensor = _w_sensortypes_select(_pd_witsml_flattened)


In [ ]:
try_sensor

In [ ]:
create_plot(_make_interactive()[0],_make_interactive()[1])

In [ ]:
# TItle of plot is not updating bc you don't trigger _create_plot(), only change the ds within it

In [ ]:
import pandas as pd
import hvplot.pandas
import param

# Define the interactive object with a select widget
class MyObject(param.Parameterized):
    select_widget = param.Selector(objects=['Option 1', 'Option 2'], default='Option 1', label='Select an option')

    def plot(self):
        # Generate a sample DataFrame
        df = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})

        # Create the plot with hvplot
        plot = df.hvplot.scatter(x='x', y='y', title=self.select_widget)

        # Watch for changes in the value of the select widget and update the title of the plot
        def update_title(event):
            plot.title = event.new
        self.param.watch(update_title, 'select_widget')

        return plot

# Create an instance of the interactive object and display the plot
my_object = MyObject()
my_object.plot()

In [ ]:
_w_wellbores_select(_pd_witsml_flattened)

In [ ]:
_w_wellbores_select(_pd_witsml_flattened).param.values()

In [ ]:


df = pn.rx(data)
plot = df.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['Date'], 
    ylim=(min(df['Date']), max(df['Date']))    # ).opts(invert_yaxis=True)
)



In [ ]:
df = data
e = pn.widgets.IntSlider(value=2, start=1, end=10, name="e").rx()
pn.Row(pn.rx(df.hvplot.line)(line_width=e))

In [ ]:
min(data['Date'])

In [ ]:
create_plot(data, _pd_witsml_flattened)

In [ ]:
print(wellbore, sensor)

In [ ]:
_get_sensorid(_pd_witsml_flattened)

In [ ]:
_pd_witsml_flattened[(_pd_witsml_flattened['nameWellbore'] == 'NO 16/2-D-18') & (_pd_witsml_flattened['uid'] == 'BITDEP')]


In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['id'] == 881650803765]

In [ ]:
_pd_witsml_flattened

In [ ]:
_sel_sensorid

In [ ]:
id, data = _make_pd_real_time_plot(7119861774595595)
data

In [ ]:
test= pn.rx(_pd_real_time_plot)

head_widget = pn.widgets.IntSlider(name="Head", start=1, end=10)

test.head(head_widget)

In [ ]:

pn.ReactiveExpr(
    test.head(head_widget),  # only show a few rows to save some space
    widget_layout=pn.Column,
    widget_location="top",
)

In [ ]:
table = pn.widgets.Tabulator(test, page_size=10, pagination="remote")
pn.Column(_w_wellbores_select(_pd_witsml),  table)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st

from matplotlib.figure import Figure

data = np.random.normal(1, 1, size=100)
fig = Figure(figsize=(8,4))
ax = fig.subplots()
bins = pn.widgets.slider(value=20, min_value=10, max_value=30, step=1, label="Bins")
ax.hist(data, bins=bins)

st.pyplot(fig)